## 4.1.1 Comparing Clocks

不同平台的时钟之实现细节不同（感觉不重要）

In [1]:
# CODE LIST 4-1
import textwrap
import time

available_clocks = [
    ('monotonic', time.monotonic),
    ('perf_counter', time.perf_counter),
    ('process_time', time.process_time),
    ('time', time.time),
]

for clock_name, func in available_clocks:
    print(textwrap.dedent('''\
    {name}:
        adjustable    : {info.adjustable}
        implementation: {info.implementation}
        monotonic     : {info.monotonic}
        resolution    : {info.resolution}
        current       : {current}
    ''').format(
        name=clock_name,
        info=time.get_clock_info(clock_name),
        current=func())
    )

monotonic:
    adjustable    : False
    implementation: GetTickCount64()
    monotonic     : True
    resolution    : 0.015625
    current       : 599077.093

perf_counter:
    adjustable    : False
    implementation: QueryPerformanceCounter()
    monotonic     : True
    resolution    : 1e-07
    current       : 0.9617127

process_time:
    adjustable    : False
    implementation: GetProcessTimes()
    monotonic     : True
    resolution    : 1e-07
    current       : 0.796875

time:
    adjustable    : True
    implementation: GetSystemTimeAsFileTime()
    monotonic     : False
    resolution    : 0.015625
    current       : 1651047340.89023



## 4.1.2 Wall Clock Time

- ***.time( ) -> float*** 当前时间的 *纪元秒数*

- ***.ctime([sec]) -> str*** 人类可读时间，缺省则为当前时间。注意返回值是字符串类型。

In [19]:
# CODE LIST 4-2, 4-3
import time

print('The time is:', time.time())

print(type(time.ctime()))
print('The time is      :', time.ctime())
later = time.time() + 15
print('15 secs from now :', time.ctime(later))

The time is: 1651051103.8701065
<class 'str'>
The time is      : Wed Apr 27 17:18:23 2022
15 secs from now : Wed Apr 27 17:18:38 2022


## 4.1.3 Monotonic Clocks / 单调时钟

*.time( )* 查看系统时钟，而用户或系统服务的行为可能改变系统时钟以完成同步等。<br>

- ***.monotonic( )*** 总是返回向前的值，以避免向后的时间产生的错误

- ***.sleep(secs)*** 程序休眠一段时间

单调时钟的起点没有定义，只有做差等比较过程中才有意义。

In [11]:
# CODE LIST 4-4
import time

start = time.monotonic()
time.sleep(0.1)
end = time.monotonic()
print('start : {:>9.2f}'.format(start))
print('end   : {:>9.2f}'.format(end))
print('span  : {:>9.2f}'.format(end - start))

start : 600676.69
end   : 600676.81
span  :      0.12


## 4.1.4 Processor Clock Time

*.time( )* 返回 Wall Clock 的时间，而 *.clock( )* 返回处理器时钟时间。
<br> .clock 在Python3.3被废弃，在3.8被移除

- ***.process_time( )*** 返回值反映了程序运行时使用的实际时间。

Typically, the processor clock **does not tick** if a program is not doing anything.
<br>（程序不运行，则处理器时间不转）

In [ ]:
# CODE LIST 4-5
import hashlib
import time

# Data to use to calculate md5 checksums
data = open(__file__, 'rb').read()

for i in range(5):
    h = hashlib.sha1()
    print(time.ctime(), ': {:0.3f} {:0.3f}'.format(
        time.time(), time.process_time()))
    for i in range(300000):
        h.update(data)
    cksum = h.digest()

# Wed Apr 27 16:47:27 2022 : 1651049247.089 0.031
# Wed Apr 27 16:47:27 2022 : 1651049247.175 0.109
# Wed Apr 27 16:47:27 2022 : 1651049247.271 0.219
# Wed Apr 27 16:47:27 2022 : 1651049247.363 0.297
# Wed Apr 27 16:47:27 2022 : 1651049247.467 0.406

In [13]:
# CODE LIST 4-6
import time

template = '{} - {:0.2f} - {:0.2f}'

print(template.format(
    time.ctime(), time.time(), time.process_time())
)

for i in range(3, 0, -1):
    print('Sleeping', i)
    time.sleep(i)
    print(template.format(
        time.ctime(), time.time(), time.process_time())
    )

Wed Apr 27 16:49:57 2022 - 1651049397.90 - 1.39
Sleeping 3
Wed Apr 27 16:50:00 2022 - 1651049400.91 - 1.39
Sleeping 2
Wed Apr 27 16:50:02 2022 - 1651049402.91 - 1.39
Sleeping 1
Wed Apr 27 16:50:03 2022 - 1651049403.93 - 1.39


## 4.1.5 Performance Counter

测量性能时需要高分辨率时钟。

- ***.perf_counter( )***

类似于 *.monotonic( )* , *perf_counter( )* 的纪元也未定义，返回值只用于比较和计算值。

In [ ]:
# CODE LIST 4-7
import hashlib
import time

# Data to use to calculate md5 checksums
data = open(__file__, 'rb').read()

loop_start = time.perf_counter()

for i in range(5):
    iter_start = time.perf_counter()
    h = hashlib.sha1()
    for i in range(300000):
        h.update(data)
    cksum = h.digest()
    now = time.perf_counter()
    loop_elapsed = now - loop_start
    iter_elapsed = now - iter_start
    print(time.ctime(), ': {:0.3f} {:0.3f}'.format(
        iter_elapsed, loop_elapsed))

# Wed Apr 27 16:54:49 2022 : 0.113 0.113
# Wed Apr 27 16:54:50 2022 : 0.114 0.227
# Wed Apr 27 16:54:50 2022 : 0.113 0.341
# Wed Apr 27 16:54:50 2022 : 0.113 0.455
# Wed Apr 27 16:54:50 2022 : 0.113 0.567

## 4.1.6 Time Components

- ***.gmtime( ) -> struct_time*** 以 UTC 格式返回当前时间（Greenwich Mean Time）

-  ***.localtime( ) -> struct_time*** 当前时区的当前时间

-   ***.mktime( )*** .localtime的反函数，将 struct_time 格式时间转换为纪元秒数

time 模块定义了 struct_time 来保存日期和时间值，分解各个组成部分以便于访问。

In [15]:
# CODE LIST 4-8
import time

def show_struct(s):
    print('  tm_year :', s.tm_year)
    print('  tm_mon  :', s.tm_mon)
    print('  tm_mday :', s.tm_mday)
    print('  tm_hour :', s.tm_hour)
    print('  tm_min  :', s.tm_min)
    print('  tm_sec  :', s.tm_sec)
    print('  tm_wday :', s.tm_wday)
    print('  tm_yday :', s.tm_yday)
    print('  tm_isdst:', s.tm_isdst)

print('gmtime:')
show_struct(time.gmtime())
print('\nlocaltime:')
show_struct(time.localtime())

print('\nlocaltime:', time.localtime())
print('\nmktime:', time.mktime(time.localtime()))

gmtime:
  tm_year : 2022
  tm_mon  : 4
  tm_mday : 27
  tm_hour : 9
  tm_min  : 8
  tm_sec  : 40
  tm_wday : 2
  tm_yday : 117
  tm_isdst: 0

localtime:
  tm_year : 2022
  tm_mon  : 4
  tm_mday : 27
  tm_hour : 17
  tm_min  : 8
  tm_sec  : 40
  tm_wday : 2
  tm_yday : 117
  tm_isdst: 0

localtime: time.struct_time(tm_year=2022, tm_mon=4, tm_mday=27, tm_hour=17, tm_min=8, tm_sec=40, tm_wday=2, tm_yday=117, tm_isdst=0)

mktime: 1651050520.0


## 4.1.7 Working with Time Zones

windows 下没有提供time.tzset() 所以本小节废弃

In [ ]:
# CODE LIST 4-9
import time
import os


def show_zone_info():
    print('  TZ    :', os.environ.get('TZ', '(not set)'))
    print('  tzname:', time.tzname)
    print('  Zone  : {} ({})'.format(
        time.timezone, (time.timezone / 3600)))
    print('  DST   :', time.daylight)
    print('  Time  :', time.ctime())
    print()


print('Default :')
show_zone_info()

ZONES = [
    'GMT',
    'Europe/Amsterdam',
]

for zone in ZONES:
    os.environ['TZ'] = zone
    time.tzset() 
    print(zone, ':')
    show_zone_info()

# AttributeError: module 'time' has no attribute 'tzset'
# Windows 系统好像不可使用 tzset

## 4.1.8 Parsing and Formatting Times
解析和格式化时间

- ***.strptime(str) -> struct_time*** 解析时间，将str类型时间转换为struct_time类型

- ***.strftime(struct_time) -> str*** 与上相反

.ctime() 的返回值就是 str 类型时间，注意格式

In [18]:
# CODE LIST 4-10
import time

def show_struct(s):
    print('  tm_year :', s.tm_year)
    print('  tm_mon  :', s.tm_mon)
    print('  tm_mday :', s.tm_mday)
    print('  tm_hour :', s.tm_hour)
    print('  tm_min  :', s.tm_min)
    print('  tm_sec  :', s.tm_sec)
    print('  tm_wday :', s.tm_wday)
    print('  tm_yday :', s.tm_yday)
    print('  tm_isdst:', s.tm_isdst)

now = time.ctime(1483391847.433716)
print('Now:', now)

parsed = time.strptime(now)
print('\nParsed:')
show_struct(parsed)

print('\nFormatted:',
      time.strftime("%a %b %d %H:%M:%S %Y", parsed))

Now: Tue Jan  3 05:17:27 2017

Parsed:
  tm_year : 2017
  tm_mon  : 1
  tm_mday : 3
  tm_hour : 5
  tm_min  : 17
  tm_sec  : 27
  tm_wday : 1
  tm_yday : 3
  tm_isdst: -1

Formatted: Tue Jan 03 05:17:27 2017
